<a href="https://colab.research.google.com/github/Athugodage/for_inca/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We created the following dataset [here](https://github.com/Athugodage/for_inca/blob/main/choose_pipeline.ipynb) 

In the same notebook we found out that transformers fine-tuned "distilbert-base-uncased" model is the best (for us). 


In [10]:
import pandas as pd


df = pd.read_csv('for_inca.csv')
df

,App name,Username,Timestamp,App review text,Sentiment score,country (language),Marketplace
0,"Crypto.com - Buy BTC, ETH",BKO,2022-06-18 18:10:03,I've had a great experience with this app. At ...,NaN,NaN,Google Play Store
1,"Crypto.com - Buy BTC, ETH",Ryan Jessup,2022-05-31 18:12:34,Opened my ap this morning and it just keeps te...,NaN,NaN,Google Play Store
2,"Crypto.com - Buy BTC, ETH",Blake Kent,2021-06-17 07:49:31,I'm not quite sure why there are SO MANY bad r...,NaN,NaN,Google Play Store
3,"Crypto.com - Buy BTC, ETH",Tooth Picks,2021-11-09 11:52:13,"It has its problems, mainly on the fact that i...",NaN,NaN,Google Play Store
4,"Crypto.com - Buy BTC, ETH",Adam Cooper,2021-03-26 11:28:52,I personally have had a great experience so fa...,NaN,NaN,Google Play Store
...,...,...,...,...,...,...,...
1367,"Binance: BTC, 600+ Crypto, NFT",chandni0,2022-04-24 16:03:45,Used so many other trading apps and Binance is...,NaN,NaN,Google Play Store
1368,"Binance: BTC, 600+ Crypto, NFT",Peter Arthold,2022-05-09 12:14:56,Best Crypto Trading Platform. I have been oper...,NaN,NaN,Google Play Store
1369,"Binance: BTC, 600+ Crypto, NFT",C.X,2022-06-29 11:38:53,"Best Application, easy to use, I hope there ar...",NaN,NaN,Google Play Store
1370,"Binance: BTC, 600+ Crypto, NFT",Beelze,2022-06-29 06:14:10,this is my first crypto trading app and have a...,NaN,NaN,Google Play Store


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.5 MB/s 
     |████████████████████████████████| 120 kB 66.5 MB/s 
     |████████████████████████████████| 212 kB 61.7 MB/s 
     |████████████████████████████████| 115 kB 70.8 MB/s 
     |████████████████████████████████| 127 kB 68.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset = imdb["train"].shuffle(seed=42)
test_dataset = imdb["test"].shuffle(seed=42)
print(train_dataset[0])
print(test_dataset[0])

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}
{'text': "<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<br /><br />Very quickly, 

In [ ]:
print(train_dataset['text'][:3])

['There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on the moon" It reminds me

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 44.9 MB/s 


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "model_for_inca"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/marcus2000/model_for_inca into local empty directory.


Download file pytorch_model.bin:   0%|          | 4.24k/255M [00:00<?, ?B/s]

Download file runs/Aug28_18-56-18_b20a825e9da0/events.out.tfevents.1661712983.b20a825e9da0.72.0:  42%|####1   …

Download file runs/Aug28_18-56-18_b20a825e9da0/1661712983.0155144/events.out.tfevents.1661712983.b20a825e9da0.…

Download file training_args.bin: 100%|##########| 3.30k/3.30k [00:00<?, ?B/s]

...

Clean file runs/Aug28_18-56-18_b20a825e9da0/events.out.tfevents.1661712983.b20a825e9da0.72.0:  21%|##        |…

Clean file runs/Aug28_18-56-18_b20a825e9da0/events.out.tfevents.1661715967.b20a825e9da0.72.2: 100%|##########|…

Clean file training_args.bin:  30%|###       | 1.00k/3.30k [00:00<?, ?B/s]

Clean file runs/Aug28_18-56-18_b20a825e9da0/1661712983.0155144/events.out.tfevents.1661712983.b20a825e9da0.72.…

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3126


Step,Training Loss
500,0.122900
1000,0.105700
1500,0.099700
2000,0.054800
2500,0.059900
3000,0.072400


Saving model checkpoint to model_for_inca/checkpoint-1563
Configuration saved in model_for_inca/checkpoint-1563/config.json
Model weights saved in model_for_inca/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in model_for_inca/checkpoint-1563/tokenizer_config.json
Special tokens file saved in model_for_inca/checkpoint-1563/special_tokens_map.json
tokenizer config file saved in model_for_inca/tokenizer_config.json
Special tokens file saved in model_for_inca/special_tokens_map.json
Saving model checkpoint to model_for_inca/checkpoint-3126
Configuration saved in model_for_inca/checkpoint-3126/config.json
Model weights saved in model_for_inca/checkpoint-3126/pytorch_model.bin
tokenizer config file saved in model_for_inca/checkpoint-3126/tokenizer_config.json
Special tokens file saved in model_for_inca/checkpoint-3126/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3126, training_loss=0.08590276898745917, metrics={'train_runtime': 2522.6961, 'train_samples_per_second': 19.82, 'train_steps_per_second': 1.239, 'total_flos': 6559874582978592.0, 'train_loss': 0.08590276898745917, 'epoch': 2.0})

In [ ]:
eval_transformers = trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16


In [ ]:
trainer.push_to_hub()

Saving model checkpoint to model_for_inca
Configuration saved in model_for_inca/config.json
Model weights saved in model_for_inca/pytorch_model.bin
tokenizer config file saved in model_for_inca/tokenizer_config.json
Special tokens file saved in model_for_inca/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/255M [00:00<?, ?B/s]

Upload file runs/Aug28_20-28-41_b20a825e9da0/events.out.tfevents.1661718609.b20a825e9da0.72.3:  69%|######9   …

Upload file runs/Aug28_20-28-41_b20a825e9da0/events.out.tfevents.1661721601.b20a825e9da0.72.5: 100%|##########…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/model_for_inca
   c866c88..a1e069e  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/model_for_inca
   c866c88..a1e069e  main -> main

To https://huggingface.co/marcus2000/model_for_inca
   a1e069e..a94f4f2  main -> main

   a1e069e..a94f4f2  main -> main



'https://huggingface.co/marcus2000/model_for_inca/commit/a1e069eff96c797807f46253b44ded8fa63820d5'

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained("marcus2000/model_for_inca")

In [7]:
# model = https://huggingface.co/marcus2000/model_for_inca/commit/a1e069eff96c797807f46253b44ded8fa63820d5

from transformers import pipeline

sentiment_model = pipeline(model="marcus2000/model_for_inca")

Downloading:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

In [ ]:
print(df['App review text'].to_list()[:3])

["I've had a great experience with this app. At first customer service was a bit slow in getting my bank verified but they are very good compared to other exchanges. The app works fine for me. Slightly buggy on loading balances but it's been great overall. I love the metal cars too. Sucks that they slashed rewards but still beats my bank by a long shot.", "Opened my ap this morning and it just keeps telling me to download the latest version, then it opens the play store and I have the latest version. I cannot access my account now!! So frustrating!!!! - responding to their reply. How exactly am I supposed to use your in-app chat feature when it won't open. It just loops from login to download new version to play store and then repeats Update: the new update fixed all issues. The app works pretty well now, no issues as of the last year or so now.", "I'm not quite sure why there are SO MANY bad reviews for this app, but personally I haven't had a single issue at all with this company, an

In [11]:
result = sentiment_model(df['App review text'].to_list())

In [16]:
for n, r in enumerate(result[:50]):
    if r['label'] == 'LABEL_0':
        print(df['App review text'].to_list()[n],  '\n ---> \n', r)

# here we can see all negative elements in the first 50 reviews slice

It has its problems, mainly on the fact that it's growing so fast, its easy to use, and most of these one stars are just freaking out. What I put in has gone up 4x. I can sell and trade in the system to build. Plus you can get an external wallet and move assets to another wallet if you wanna jump some fees. Slow down gove it a chance. Ohh and the CRO is great for staking, can't wait for that metal card, sure airports will love it lol 
 ---> 
 {'label': 'LABEL_0', 'score': 0.9912109375}
I'm also having trouble with the 2FA. I can't even open the app anymore. It's been 3 or 4 days now and I can't do open the app at all and customer service is super slow. I used to love the app. Not anymore. Edit: Since the updates, I've fallen back in love with the app. 
 ---> 
 {'label': 'LABEL_0', 'score': 0.9950438737869263}
Why so many issues logging in. This app crashes constantly. It's easy to use when it's actually working. This is unacceptable when you could be losing money do to bugs in the app.

In [17]:
for n, r in enumerate(result[:50]):
    if r['label'] == 'LABEL_1':
        print(df['App review text'].to_list()[n],  '\n ---> \n', r)

# and the positive ones

I've had a great experience with this app. At first customer service was a bit slow in getting my bank verified but they are very good compared to other exchanges. The app works fine for me. Slightly buggy on loading balances but it's been great overall. I love the metal cars too. Sucks that they slashed rewards but still beats my bank by a long shot. 
 ---> 
 {'label': 'LABEL_1', 'score': 0.9959709048271179}
Opened my ap this morning and it just keeps telling me to download the latest version, then it opens the play store and I have the latest version. I cannot access my account now!! So frustrating!!!! - responding to their reply. How exactly am I supposed to use your in-app chat feature when it won't open. It just loops from login to download new version to play store and then repeats Update: the new update fixed all issues. The app works pretty well now, no issues as of the last year or so now. 
 ---> 
 {'label': 'LABEL_1', 'score': 0.9396968483924866}
I'm not quite sure why there 

In [20]:
df['App review text'][0:10]

0    I've had a great experience with this app. At ...
1    Opened my ap this morning and it just keeps te...
2    I'm not quite sure why there are SO MANY bad r...
3    It has its problems, mainly on the fact that i...
4    I personally have had a great experience so fa...
5    Been great so far! There are a lot of coins on...
6    All good, I've been using the app for over a y...
7    Exceptional in every way. Just be sure to impo...
8    Easy to use. Great customer support that you c...
9    Easy to use. Plenty of info available regardin...
Name: App review text, dtype: object

In [29]:

for n, r in enumerate(result):
    if r['label'] == 'LABEL_1':
        df['Sentiment score'][n] = r['score']
    else:
        df['Sentiment score'][n] = float(float(r['score']) * -1)

        # if a text is negative, then we show a score in minus



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [32]:
df.to_csv('data_with_sentiment.csv')

In [30]:
df.head(20)

,App name,Username,Timestamp,App review text,Sentiment score,country (language),Marketplace
0,"Crypto.com - Buy BTC, ETH",BKO,2022-06-18 18:10:03,I've had a great experience with this app. At ...,0.995971,NaN,Google Play Store
1,"Crypto.com - Buy BTC, ETH",Ryan Jessup,2022-05-31 18:12:34,Opened my ap this morning and it just keeps te...,0.939697,NaN,Google Play Store
2,"Crypto.com - Buy BTC, ETH",Blake Kent,2021-06-17 07:49:31,I'm not quite sure why there are SO MANY bad r...,0.996500,NaN,Google Play Store
3,"Crypto.com - Buy BTC, ETH",Tooth Picks,2021-11-09 11:52:13,"It has its problems, mainly on the fact that i...",-0.991211,NaN,Google Play Store
4,"Crypto.com - Buy BTC, ETH",Adam Cooper,2021-03-26 11:28:52,I personally have had a great experience so fa...,0.996602,NaN,Google Play Store
5,"Crypto.com - Buy BTC, ETH",Jonathan Gentry,2021-02-18 18:55:20,Been great so far! There are a lot of coins on...,0.995850,NaN,Google Play Store
6,"Crypto.com - Buy BTC, ETH",tudor s,2022-08-17 20:28:07,"All good, I've been using the app for over a y...",0.992833,NaN,Google Play Store
7,"Crypto.com - Buy BTC, ETH",Christopher Fallucca,2022-08-23 13:13:55,Exceptional in every way. Just be sure to impo...,0.995712,NaN,Google Play Store
8,"Crypto.com - Buy BTC, ETH",Jesse Theodore,2022-08-23 22:47:28,Easy to use. Great customer support that you c...,0.996729,NaN,Google Play Store
9,"Crypto.com - Buy BTC, ETH",Merv d'Perv Productions,2022-08-17 19:26:59,Easy to use. Plenty of info available regardin...,0.987234,NaN,Google Play Store
